In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

## SinergyData
carga de dataset

In [32]:
california = pd.read_csv("../reseñas california/review_california.csv")
las_vegas = pd.read_csv("../reseñas las vegas/review_las_vegas.csv")
nueva_york= pd.read_csv("../reseñas ny/review_ny.csv")


descripcion de datos 


In [33]:
california.head(5)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.089268e+20,Christopher Thomas Constantino,2021-06-12 23:27:17.434,4,Great pet friendly place on the American River...,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1623778784860, 'text': 'Thank you for...",0x809a55833fbe05ad:0xee6507787fb6a26c
1,1.147835e+20,Valley Girl,2020-11-06 01:35:04.569,5,Beautiful outdoorsy getaway. Kitchenette cotta...,NaN,"{'time': 1610999719328, 'text': 'Thank you Val...",0x809a55833fbe05ad:0xee6507787fb6a26c
2,1.158307e+20,John Barringer,2020-11-01 03:06:47.321,5,I was the chef at this evening's wedding. I wa...,NaN,"{'time': 1604207570230, 'text': 'Thank you Joh...",0x809a55833fbe05ad:0xee6507787fb6a26c
3,1.083060e+20,Missey Iverson,2021-06-08 15:35:00.137,5,I am trying to book a small cabin. Help. 😆,NaN,"{'time': 1623178567495, 'text': ""Well this is ...",0x809a55833fbe05ad:0xee6507787fb6a26c
4,1.165939e+20,Larry S.,2018-04-09 16:11:16.898,5,My kids and I spent part of last week in Colom...,NaN,"{'time': 1592523694586, 'text': 'Thank you Lar...",0x809a55833fbe05ad:0xee6507787fb6a26c


In [34]:
las_vegas.head(5)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.064355e+20,A Hard Worker,2021-02-18 16:40:51.812,5,"Very unique place! Clean, pleasant and efficie...",NaN,NaN,0x80c8c377584fbfcf:0x73336cb8a6e11a6d
1,1.175568e+20,Greg Evans,2020-02-15 00:22:45.364,5,We had a gr8 time meeting all the We Grow Wome...,NaN,NaN,0x80c8c377584fbfcf:0x73336cb8a6e11a6d
2,1.052251e+20,Trina p,2018-08-29 15:46:28.137,4,Very unique place with 5 different themed rooms.,NaN,NaN,0x80c8c377584fbfcf:0x73336cb8a6e11a6d
3,1.156929e+20,Julie Weskamp,2018-08-22 00:04:45.556,5,"Super cute, unique. Off the strip fun.",NaN,NaN,0x80c8c377584fbfcf:0x73336cb8a6e11a6d
4,1.035557e+20,the chef garza,2020-02-14 03:59:35.128,5,Best ever!!!,NaN,NaN,0x80c8c377584fbfcf:0x73336cb8a6e11a6d


In [14]:
nueva_york.head(5)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.040650e+20,Katie K,2021-07-07 10:34:18.257,5,If I could give this place even more stars I w...,NaN,"{'time': 1625669318347, 'text': 'Thanks so muc...",0x89d486fb46b35e0f:0xf89fc350fa1c93f0
1,1.156591e+20,Nick Nyhan,2020-11-09 15:36:18.554,5,Pulled in there after Niagara Falls visit. Rea...,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1604949989257, 'text': 'Thanks for th...",0x89d486fb46b35e0f:0xf89fc350fa1c93f0
2,1.005289e+20,JustGoFlyVinnie,2020-10-26 13:20:32.178,5,I can't rate this hidden gem high enough. We g...,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1603735958781, 'text': 'Thanks so muc...",0x89d486fb46b35e0f:0xf89fc350fa1c93f0
3,1.122109e+20,Matt Carroll,2019-08-18 12:48:42.676,5,We rented the hotel for a company party for th...,NaN,"{'time': 1566149806295, 'text': 'Thanks for th...",0x89d486fb46b35e0f:0xf89fc350fa1c93f0
4,1.149143e+20,Russell Larsen,2019-01-12 11:04:50.798,5,What a Jem. Great place to host a wedding or p...,NaN,NaN,0x89d486fb46b35e0f:0xf89fc350fa1c93f0


In [35]:
# Concatenar los tres dataset en uno 
merged_reviews = pd.concat([california, las_vegas, nueva_york], ignore_index=True)
print(merged_reviews.shape)
merged_reviews.tail(10)

(6088, 8)


,user_id,name,time,rating,text,pics,resp,gmap_id
6078,1.073504e+20,Ashish Mishra,2017-09-04 00:37:53.111,3,"Rooms are clean, but small.. breakfast choice ...",NaN,NaN,0x89dfce4984aaa485:0xacbd33cecf7d50d4
6079,1.086090e+20,Harry E,2017-08-22 17:48:07.673,3,Seems to be a decent place but kind of pricey ...,NaN,NaN,0x89dfce4984aaa485:0xacbd33cecf7d50d4
6080,1.025835e+20,Deneen Homer,2016-10-12 17:33:52.337,2,Horrible customer service...minimum clean,NaN,NaN,0x89dfce4984aaa485:0xacbd33cecf7d50d4
6081,1.144082e+20,Anumit Sasidharan,2016-05-30 07:31:40.850,4,Reasonably clean and good amenities for the pr...,NaN,NaN,0x89dfce4984aaa485:0xacbd33cecf7d50d4
6082,1.141696e+20,Manuel Medina,2016-04-26 07:42:28.910,4,Nice rooms,NaN,NaN,0x89dfce4984aaa485:0xacbd33cecf7d50d4
6083,1.127285e+20,Don Todd,2018-03-20 02:12:33.163,1,NaN,NaN,NaN,0x89dfce4984aaa485:0xacbd33cecf7d50d4
6084,1.040970e+20,Luc “190falcon” Blanchette,2017-02-05 12:06:51.870,3,NaN,NaN,NaN,0x89dfce4984aaa485:0xacbd33cecf7d50d4
6085,1.032762e+20,Kathy Reeks,2018-02-26 14:43:08.810,5,NaN,NaN,NaN,0x89dfce4984aaa485:0xacbd33cecf7d50d4
6086,1.028247e+20,Matthew Gordon,2018-03-20 06:25:39.435,3,NaN,NaN,NaN,0x89dfce4984aaa485:0xacbd33cecf7d50d4
6087,1.029704e+20,Charlene Cook,2017-10-07 08:14:14.038,1,NaN,NaN,NaN,0x89dfce4984aaa485:0xacbd33cecf7d50d4


In [36]:
merged_reviews.columns

Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id'], dtype='object')

- user_id = id del usuario que hace la reseña
- name = nombre del usuario
- time = fecha en que se realizo la reseña
- rating = calificación dada por el usuario del 1 al 5
- text = reseña del usuario
- pics = url de las fotos que acompañan a la reseña
- resp = respuesta de la reseña por parte del negocio, contiene de forma anidada una fecha y un texto
- gmap_id = id del mapa

In [37]:
# informacion de las columnas 
merged_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6088 entries, 0 to 6087
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  6088 non-null   float64
 1   name     6088 non-null   object 
 2   time     6088 non-null   object 
 3   rating   6088 non-null   int64  
 4   text     3970 non-null   object 
 5   pics     202 non-null    object 
 6   resp     383 non-null    object 
 7   gmap_id  6088 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 380.6+ KB


Se pueden observar que en la columna de text, pics y resp hacen falta datos por lo que debemos normalizarlos 

In [53]:
#recuento de nulos
total_datos=merged_reviews.shape[0]
total_no_nulos_text = total_datos - merged_reviews["text"].isnull().sum()
total_no_nulos_pics = total_datos - merged_reviews["pics"].isnull().sum()
total_no_nulos_resp = total_datos - merged_reviews["resp"].isnull().sum()
total_no_nulos_name = total_datos - merged_reviews["name"].isnull().sum()
total_no_nulos_time = total_datos - merged_reviews["time"].isnull().sum()
total_no_nulos_rating = total_datos - merged_reviews["rating"].isnull().sum()
total_no_nulos_gmap_id = total_datos - merged_reviews["gmap_id"].isnull().sum()


In [62]:
categorias = ['text', 'pics', 'resp', 'name', 'time', 'rating','gmai_id']
valores = [total_no_nulos_text, total_no_nulos_pics, total_no_nulos_resp,total_no_nulos_name, total_no_nulos_time, total_no_nulos_rating, total_no_nulos_gmap_id]

# Colores para las barras (uno por cada barra)
colores_barras = ['#4D77FF', '#56BBF1', '#5EE6EB', 'orange', 'orange', 'orange', 'orange']

# Crear el gráfico de barras
fig = go.Figure(data=[go.Bar(x=categorias, y=valores, marker_color=colores_barras)])


# Personalizar el diseño del gráfico
fig.update_layout(title='Valores No Nulos', title_x=0.5,  
                  xaxis_title='Categorías',
                  yaxis_title='Valores')
fig.show()


In [ ]:
# NORMALIZAR DATOS NaN
merged_reviews["text"] = merged_reviews["text"].fillna("")
merged_reviews["pics"] = merged_reviews["pics"].fillna("")
merged_reviews["resp"] = merged_reviews["resp"].fillna("")
merged_reviews.tail(10)
